In [ ]:
%%capture
!pip install neuralforecast
!pip install hyperopt

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (9,6)
from neuralforecast.core import NeuralForecast
from neuralforecast.models import NHITS, NBEATS, TimesNet
from neuralforecast.losses.numpy import mae, mse
from neuralforecast.auto import AutoTimesNet
from neuralforecast.models import DeepAR

import logging
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Time-Series-Library-main/dataset/samsung19_27.csv')
df.rename(columns={'Datetime': 'ds'}, inplace=True)
df.rename(columns={'Close': 'y'}, inplace=True)
df['ds'] = pd.to_datetime(df['ds'])
df['unique_id'] = 'OT'

In [5]:
train_df = df.iloc[:-60,:]
futr_df = df.iloc[-60:,:]

In [7]:
futr_df

,ds,Open,High,Low,y,Adj Close,Volume,unique_id
2450,2024-03-27 14:00:00+09:00,79700.0,79700.0,79600.0,79600.0,79600.0,19945,OT
2451,2024-03-27 14:01:00+09:00,79600.0,79700.0,79600.0,79600.0,79600.0,10156,OT
2452,2024-03-27 14:02:00+09:00,79700.0,79700.0,79600.0,79700.0,79700.0,8308,OT
2453,2024-03-27 14:03:00+09:00,79700.0,79700.0,79600.0,79700.0,79700.0,14987,OT
2454,2024-03-27 14:04:00+09:00,79700.0,79700.0,79600.0,79600.0,79600.0,10376,OT
2455,2024-03-27 14:05:00+09:00,79600.0,79700.0,79600.0,79600.0,79600.0,18549,OT
2456,2024-03-27 14:06:00+09:00,79600.0,79700.0,79600.0,79600.0,79600.0,27398,OT
2457,2024-03-27 14:07:00+09:00,79700.0,79700.0,79500.0,79500.0,79500.0,337250,OT
2458,2024-03-27 14:08:00+09:00,79600.0,79600.0,79500.0,79600.0,79600.0,25292,OT
2459,2024-03-27 14:09:00+09:00,79600.0,79600.0,79500.0,79500.0,79500.0,11398,OT


In [ ]:
# optimiztion 필요
horizon = 60 # 60 minuts forecast
model = TimesNet(h = horizon,                                                 # Horizon
                 input_size = 3*horizon,                                      # Length of input window
                 max_steps = 20,                                             # Training iterations
                 top_k = 3,                                                   # Number of periods (for FFT).
                 num_kernels = 3,                                             # Number of kernels for Inception module
                 batch_size = 2,                                              # Number of time series per batch
                 windows_batch_size = 32,                                     # Number of windows per batch
                 learning_rate = 0.001,                                       # Learning rate
                 futr_exog_list = ['Open', 'High','Volume'], # Future exogenous variables
                 scaler_type = None)                                          # We use the Core scaling method

In [ ]:
nf = NeuralForecast(models=[model], freq='T', local_scaler_type='robust')
nf.fit(df=train_df)

In [ ]:
Y_hat_df = nf.predict(futr_df=futr_df)
Y_hat_df.head()

In [ ]:
import matplotlib.pyplot as plt

plot_df = train_df[train_df['unique_id']=='OT'].tail(60*3).reset_index(drop=True)
Y_hat_df = Y_hat_df.reset_index(drop=False)
Y_hat_df = Y_hat_df[Y_hat_df['unique_id']=='OT']

plot_df = pd.concat([plot_df, Y_hat_df ]).set_index('ds') # Concatenate the train and forecast dataframes

plot_df[['y', 'TimesNet']].plot(linewidth=2)
plt.axvline('2024-03-27', color='red')
plt.ylabel('Price Samsung Stock', fontsize=12)
plt.xlabel('Date', fontsize=12)
plt.grid()

In [ ]:
data = {'TimesNet': [mae(Y_hat_df['TimesNet'], Y_hat_df['y']), mse(Y_hat_df['TimesNet'], Y_hat_df['y'])]}

metrics_df = pd.DataFrame(data=data)
metrics_df.index = ['mae', 'mse']

metrics_df.style.highlight_min(color='lightgreen', axis=1)

In [ ]:
.